# Classificação de pontuação de crédito
Nomes: Carlos Bravo e Markson Arguello

### Imports

In [1074]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import pandas as pd

### Carregar dados

In [1049]:
# Lê o arquivo csv
df = pd.read_csv('dados/CreditScoreData.csv')

# Mostra as 5 primeiras linhas
df.head()

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age_Months,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,0x160f,CUS_0x21b1,February,Rick Rothackerj,28,004-07-5839,Teacher,"34,848","3,038",2,4,6,1,Credit-Builder Loan,7,1,7.42,2,Good,605.03,38.55,320,No,18.82,40.39,High_spent_Large_value_payments,484.59,Good
1,0x1612,CUS_0x21b1,May,Rick Rothackerj,28,004-07-5839,Teacher,"34,848","3,038",2,4,6,1,Credit-Builder Loan,3,1,6.42,2,Good,605.03,34.98,323,No,18.82,130.12,Low_spent_Small_value_payments,444.87,Good
2,0x1613,CUS_0x21b1,June,Rick Rothackerj,28,004-07-5839,Teacher,"34,848","3,038",2,4,6,1,Credit-Builder Loan,3,0,5.42,2,Good,605.03,33.38,324,No,18.82,43.48,High_spent_Large_value_payments,481.51,Good
3,0x1615,CUS_0x21b1,August,Rick Rothackerj,28,004-07-5839,Teacher,"34,848","3,038",2,4,6,1,Credit-Builder Loan,3,4,5.42,2,Good,605.03,32.93,326,No,18.82,218.90,Low_spent_Small_value_payments,356.08,Good
4,0x1626,CUS_0xb891,January,Jasond,54,072-31-6145,Entrepreneur,"30,690","2,612",2,5,4,1,Not Specified,0,6,1.99,4,Good,632.46,26.54,207,No,16.42,81.23,Low_spent_Large_value_payments,433.60,Standard


### Limpeza dos dados

In [1050]:
# Remover ID e dados sensíveis
df = df.drop(['ID', 'Customer_ID', 'Name', 'SSN'], axis=1)

In [1051]:
# Verifica valores nulos
lista = df.isnull().sum()

for i in range(len(lista)):
    if lista[i] > 0:
        print(lista.index[i], lista[i])

Type_of_Loan 2526


In [1052]:
# Verifica linhas repetidas
df.duplicated().sum()

0

#### Converter colunas categóricas em numéricas

#### Coluna Month

In [1053]:
# Mapeia cada mes para um numero
meses = {'January': 1, 'February': 2, 'March': 3, 'April': 4, 'May': 5, 'June': 6, 'July': 7, 'August': 8, 'September': 9, 'October': 10, 'November': 11, 'December': 12}

# Converte a coluna 'Month' para inteiro
df['Month'] = df['Month'].map(meses)


#### Coluna Credit_Mix

In [1054]:
# Pega os valores distintos da coluna 'Credit_Mix'
df['Credit_Mix'].unique()

array(['Good', 'Standard', 'Bad'], dtype=object)

In [1055]:
# Mapeia cada valor para um numero
credit_mix = {'Bad': 1, 'Standard': 2, 'Good': 3}

# Converte a coluna 'Credit_Mix' para inteiro
df['Credit_Mix'] = df['Credit_Mix'].map(credit_mix)


#### Coluna Payment_of_Min_Amount

In [1056]:
# Pega os valores distintos da coluna 'Payment_of_Min_Amount'
df['Payment_of_Min_Amount'].unique()


array(['No', 'Yes'], dtype=object)

In [1057]:
# Mapeia cada valor para um numero
payment_of_min_amount = {'Yes': 1, 'No': 0}

# Converte a coluna 'Payment_of_Min_Amount' para inteiro
df['Payment_of_Min_Amount'] = df['Payment_of_Min_Amount'].map(payment_of_min_amount)

#### Coluna Credit_Score

In [1058]:
# Pega os valores distintos da coluna 'Credit_Score'
df['Credit_Score'].unique()

array(['Good', 'Standard', 'Poor'], dtype=object)

In [1059]:
# Mapeia cada valor para um numero
credit_score = {'Poor': 1, 'Standard': 2, 'Good': 3}

# Converte a coluna 'Credit_Score' para inteiro
df['Credit_Score'] = df['Credit_Score'].map(credit_score)

#### One-Hot Encoding

In [1060]:
# Função para criar One-Hot Encoding para uma coluna
def one_hot_encoding(col):
    global df
    y = pd.get_dummies(df[col], prefix=col)
    df = pd.concat([df, y], axis=1)
    df = df.drop([col], axis=1)

In [1061]:
# Aplica a função para as colunas 'Occupation' e 'Payment_Behaviour'
one_hot_encoding('Occupation')
one_hot_encoding('Payment_Behaviour')

In [1062]:
df.head()

,Month,Age,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age_Months,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Monthly_Balance,Credit_Score,Occupation_Accountant,Occupation_Architect,Occupation_Developer,Occupation_Doctor,Occupation_Engineer,Occupation_Entrepreneur,Occupation_Journalist,Occupation_Lawyer,Occupation_Manager,Occupation_Mechanic,Occupation_Media_Manager,Occupation_Musician,Occupation_Scientist,Occupation_Teacher,Occupation_Writer,Payment_Behaviour_High_spent_Large_value_payments,Payment_Behaviour_High_spent_Medium_value_payments,Payment_Behaviour_High_spent_Small_value_payments,Payment_Behaviour_Low_spent_Large_value_payments,Payment_Behaviour_Low_spent_Medium_value_payments,Payment_Behaviour_Low_spent_Small_value_payments
0,2,28,"34,848","3,038",2,4,6,1,Credit-Builder Loan,7,1,7.42,2,3,605.03,38.55,320,0,18.82,40.39,484.59,3,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0
1,5,28,"34,848","3,038",2,4,6,1,Credit-Builder Loan,3,1,6.42,2,3,605.03,34.98,323,0,18.82,130.12,444.87,3,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
2,6,28,"34,848","3,038",2,4,6,1,Credit-Builder Loan,3,0,5.42,2,3,605.03,33.38,324,0,18.82,43.48,481.51,3,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0
3,8,28,"34,848","3,038",2,4,6,1,Credit-Builder Loan,3,4,5.42,2,3,605.03,32.93,326,0,18.82,218.90,356.08,3,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
4,1,54,"30,690","2,612",2,5,4,1,Not Specified,0,6,1.99,4,3,632.46,26.54,207,0,16.42,81.23,433.60,2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [1063]:
# Substitui nan, null por string vazia na coluna 'Type_of_Loan'
df['Type_of_Loan'] = df['Type_of_Loan'].fillna('')
df['Type_of_Loan'] = df['Type_of_Loan'].replace('null', '')

# Pega os valores da coluna 'Type_of_Loan'
lista = df['Type_of_Loan']

valores = []

for elemento in lista:
    parte = elemento.replace('and ', '').split(', ')
    valores.append(parte)

In [1064]:
# Cria um dataframe com o encode da coluna 'Type_of_Loan'
valores = pd.Series(valores, dtype='object')
encoding = pd.get_dummies(valores.apply(pd.Series, dtype='object').stack()).groupby(level=0).sum()

In [1065]:
# Remove coluna 'Not Specified' e '' 
encoding = encoding.drop(['Not Specified', ''], axis=1)

In [1066]:
# Concatena o encoding com o dataframe
df = pd.concat([df, encoding], axis=1)

# Substitui nan por 0
df = df.fillna(0)

# Remove a coluna 'Type_of_Loan'
df = df.drop(['Type_of_Loan'], axis=1)

In [1067]:
df.head()

,Month,Age,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age_Months,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Monthly_Balance,Credit_Score,Occupation_Accountant,Occupation_Architect,Occupation_Developer,Occupation_Doctor,Occupation_Engineer,Occupation_Entrepreneur,Occupation_Journalist,Occupation_Lawyer,Occupation_Manager,Occupation_Mechanic,Occupation_Media_Manager,Occupation_Musician,Occupation_Scientist,Occupation_Teacher,Occupation_Writer,Payment_Behaviour_High_spent_Large_value_payments,Payment_Behaviour_High_spent_Medium_value_payments,Payment_Behaviour_High_spent_Small_value_payments,Payment_Behaviour_Low_spent_Large_value_payments,Payment_Behaviour_Low_spent_Medium_value_payments,Payment_Behaviour_Low_spent_Small_value_payments,Auto Loan,Credit-Builder Loan,Debt Consolidation Loan,Home Equity Loan,Mortgage Loan,Payday Loan,Personal Loan,Student Loan
0,2,28,"34,848","3,038",2,4,6,1,7,1,7.42,2,3,605.03,38.55,320,0,18.82,40.39,484.59,3,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
1,5,28,"34,848","3,038",2,4,6,1,3,1,6.42,2,3,605.03,34.98,323,0,18.82,130.12,444.87,3,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0
2,6,28,"34,848","3,038",2,4,6,1,3,0,5.42,2,3,605.03,33.38,324,0,18.82,43.48,481.51,3,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
3,8,28,"34,848","3,038",2,4,6,1,3,4,5.42,2,3,605.03,32.93,326,0,18.82,218.90,356.08,3,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0
4,1,54,"30,690","2,612",2,5,4,1,0,6,1.99,4,3,632.46,26.54,207,0,16.42,81.23,433.60,2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


#### Converte colunas com números de formato string para float

In [1068]:
# Função para convertr a coluna para float
def converte_para_float(col):
    global df
    df[col].replace('[, ]', '', regex=True, inplace=True)
    df[col] = df[col].astype(float)

In [1069]:
for col in df.columns:
    if df[col].dtype == 'object':
        converte_para_float(col)

In [1070]:
df.head()

,Month,Age,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age_Months,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Monthly_Balance,Credit_Score,Occupation_Accountant,Occupation_Architect,Occupation_Developer,Occupation_Doctor,Occupation_Engineer,Occupation_Entrepreneur,Occupation_Journalist,Occupation_Lawyer,Occupation_Manager,Occupation_Mechanic,Occupation_Media_Manager,Occupation_Musician,Occupation_Scientist,Occupation_Teacher,Occupation_Writer,Payment_Behaviour_High_spent_Large_value_payments,Payment_Behaviour_High_spent_Medium_value_payments,Payment_Behaviour_High_spent_Small_value_payments,Payment_Behaviour_Low_spent_Large_value_payments,Payment_Behaviour_Low_spent_Medium_value_payments,Payment_Behaviour_Low_spent_Small_value_payments,Auto Loan,Credit-Builder Loan,Debt Consolidation Loan,Home Equity Loan,Mortgage Loan,Payday Loan,Personal Loan,Student Loan
0,2,28,34848.0,3038.0,2,4,6,1,7,1,7.42,2,3,605.03,38.55,320,0,18.82,40.39,484.59,3,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
1,5,28,34848.0,3038.0,2,4,6,1,3,1,6.42,2,3,605.03,34.98,323,0,18.82,130.12,444.87,3,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0
2,6,28,34848.0,3038.0,2,4,6,1,3,0,5.42,2,3,605.03,33.38,324,0,18.82,43.48,481.51,3,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
3,8,28,34848.0,3038.0,2,4,6,1,3,4,5.42,2,3,605.03,32.93,326,0,18.82,218.90,356.08,3,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0
4,1,54,30690.0,2612.0,2,5,4,1,0,6,1.99,4,3,632.46,26.54,207,0,16.42,81.23,433.60,2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


### Normalização e padronização dos dados

In [1071]:
colunas_para_conversao = [
    "Age",
    "Annual_Income",
    "Monthly_Inhand_Salary",
    "Num_Bank_Accounts",
    "Num_Credit_Card",
    "Interest_Rate",
    "Num_of_Loan",
    "Delay_from_due_date",
    "Num_of_Delayed_Payment",
    "Changed_Credit_Limit",
    "Num_Credit_Inquiries",
    "Credit_Mix",
    "Outstanding_Debt",
    "Credit_Utilization_Ratio",
    "Credit_History_Age_Months",
    "Payment_of_Min_Amount",
    "Total_EMI_per_month",
    "Amount_invested_monthly",
    "Monthly_Balance"
]

In [1072]:
# Normaliza as colunas selecionadas
scaler = MinMaxScaler()
df_normalizado = df.copy()
df_normalizado[colunas_para_conversao] = scaler.fit_transform(df[colunas_para_conversao])


df_normalizado.head(10)

,Month,Age,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age_Months,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Monthly_Balance,Credit_Score,Occupation_Accountant,Occupation_Architect,Occupation_Developer,Occupation_Doctor,Occupation_Engineer,Occupation_Entrepreneur,Occupation_Journalist,Occupation_Lawyer,Occupation_Manager,Occupation_Mechanic,Occupation_Media_Manager,Occupation_Musician,Occupation_Scientist,Occupation_Teacher,Occupation_Writer,Payment_Behaviour_High_spent_Large_value_payments,Payment_Behaviour_High_spent_Medium_value_payments,Payment_Behaviour_High_spent_Small_value_payments,Payment_Behaviour_Low_spent_Large_value_payments,Payment_Behaviour_Low_spent_Medium_value_payments,Payment_Behaviour_Low_spent_Small_value_payments,Auto Loan,Credit-Builder Loan,Debt Consolidation Loan,Home Equity Loan,Mortgage Loan,Payday Loan,Personal Loan,Student Loan
0,2,0.333333,0.001151,0.183478,0.111111,0.333333,0.066667,0.111111,0.104478,0.000227,0.324362,0.000771,1.0,0.120993,0.616109,0.791045,0.0,0.000229,0.024532,0.309126,3,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
1,5,0.333333,0.001151,0.183478,0.111111,0.333333,0.066667,0.111111,0.044776,0.000227,0.300960,0.000771,1.0,0.120993,0.491632,0.798507,0.0,0.000229,0.079034,0.283764,3,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0
2,6,0.333333,0.001151,0.183478,0.111111,0.333333,0.066667,0.111111,0.044776,0.000000,0.277557,0.000771,1.0,0.120993,0.435844,0.800995,0.0,0.000229,0.026409,0.307159,3,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
3,8,0.333333,0.001151,0.183478,0.111111,0.333333,0.066667,0.111111,0.044776,0.000910,0.277557,0.000771,1.0,0.120993,0.420153,0.805970,0.0,0.000229,0.132958,0.227071,3,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0
4,1,0.952381,0.000979,0.154889,0.111111,0.444444,0.040000,0.111111,0.000000,0.001365,0.197285,0.001542,1.0,0.126481,0.197350,0.509950,0.0,0.000200,0.049338,0.276568,2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
5,2,0.952381,0.000979,0.154889,0.111111,0.444444,0.040000,0.111111,0.074627,0.000682,0.197285,0.001542,1.0,0.126481,0.502092,0.512438,0.0,0.000200,0.075851,0.261468,2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
6,3,0.976190,0.000979,0.154889,0.111111,0.444444,0.040000,0.111111,0.044776,0.002047,0.197285,0.001542,1.0,0.126481,0.398187,0.514925,0.0,0.000200,0.050662,0.262413,2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
7,7,0.166667,0.001180,0.171062,0.666667,0.444444,0.053333,0.000000,0.149254,0.003411,0.211093,0.001542,0.5,0.188790,0.187587,0.925373,1.0,0.000000,0.109944,0.251488,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
8,4,0.404762,0.002766,0.381518,0.333333,0.444444,0.093333,0.000000,0.119403,0.001592,0.388018,0.000771,1.0,0.109622,0.324965,0.955224,0.0,0.000000,0.375488,0.172537,3,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
9,5,0.214286,0.004458,0.640225,0.111111,0.444444,0.080000,0.333333,0.164179,0.002502,0.413761,0.003084,1.0,0.275605,0.311715,0.641791,0.0,0.002760,0.394688,0.234478,2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0


In [1076]:
# Padroniza as colunas selecionadas
scaler = StandardScaler()
df_padronizado = df.copy()
df_padronizado[colunas_para_conversao] = scaler.fit_transform(df[colunas_para_conversao])

df_padronizado.head(10)

,Month,Age,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age_Months,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Monthly_Balance,Credit_Score,Occupation_Accountant,Occupation_Architect,Occupation_Developer,Occupation_Doctor,Occupation_Engineer,Occupation_Entrepreneur,Occupation_Journalist,Occupation_Lawyer,Occupation_Manager,Occupation_Mechanic,Occupation_Media_Manager,Occupation_Musician,Occupation_Scientist,Occupation_Teacher,Occupation_Writer,Payment_Behaviour_High_spent_Large_value_payments,Payment_Behaviour_High_spent_Medium_value_payments,Payment_Behaviour_High_spent_Small_value_payments,Payment_Behaviour_Low_spent_Large_value_payments,Payment_Behaviour_Low_spent_Medium_value_payments,Payment_Behaviour_Low_spent_Small_value_payments,Auto Loan,Credit-Builder Loan,Debt Consolidation Loan,Home Equity Loan,Mortgage Loan,Payday Loan,Personal Loan,Student Loan
0,2,-0.472322,-0.097249,-0.329689,-1.544442,-0.785454,-1.030988,-1.070128,-0.999420,-0.133966,-0.482674,-0.133669,1.381380,-0.736882,1.231717,1.028402,-1.126886,-0.167676,-0.797519,0.432906,3,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
1,5,-0.472322,-0.097249,-0.329689,-1.544442,-0.785454,-1.030988,-1.070128,-1.268561,-0.133966,-0.628731,-0.133669,1.381380,-0.736882,0.527170,1.058513,-1.126886,-0.167676,-0.297787,0.240100,3,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0
2,6,-0.472322,-0.097249,-0.329689,-1.544442,-0.785454,-1.030988,-1.070128,-1.268561,-0.138210,-0.774788,-0.133669,1.381380,-0.736882,0.211406,1.068549,-1.126886,-0.167676,-0.780310,0.417956,3,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
3,8,-0.472322,-0.097249,-0.329689,-1.544442,-0.785454,-1.030988,-1.070128,-1.268561,-0.121233,-0.774788,-0.133669,1.381380,-0.736882,0.122598,1.088623,-1.126886,-0.167676,0.196654,-0.190898,3,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0
4,1,1.945399,-0.100108,-0.467344,-1.544442,-0.297915,-1.259128,-1.070128,-1.470417,-0.112745,-1.275764,-0.122921,1.381380,-0.713635,-1.138483,-0.105757,-1.126886,-0.167966,-0.570069,0.185394,2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
5,2,1.945399,-0.100108,-0.467344,-1.544442,-0.297915,-1.259128,-1.070128,-1.133991,-0.125477,-1.275764,-0.122921,1.381380,-0.713635,0.586375,-0.095720,-1.126886,-0.167966,-0.326970,0.070594,2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
6,3,2.038389,-0.100108,-0.467344,-1.544442,-0.297915,-1.259128,-1.070128,-1.268561,-0.100012,-1.275764,-0.122921,1.381380,-0.713635,-0.001734,-0.085683,-1.126886,-0.167966,-0.557928,0.077778,2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
7,7,-1.123247,-0.096768,-0.389469,0.572659,-0.297915,-1.145058,-1.478268,-0.797565,-0.074547,-1.189590,-0.122921,-0.014525,-0.449720,-1.193741,1.570389,0.887401,-0.169953,-0.014366,-0.005276,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
8,4,-0.193354,-0.070385,0.623881,-0.697602,-0.297915,-0.802848,-1.478268,-0.932135,-0.108500,-0.085399,-0.133669,1.381380,-0.785046,-0.416174,1.690831,-1.126886,-0.169953,2.420470,-0.605489,3,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
9,5,-0.937269,-0.042262,1.869564,-1.544442,-0.297915,-0.916918,-0.253849,-0.730279,-0.091524,0.075264,-0.101426,1.381380,-0.082002,-0.491168,0.426194,-1.126886,-0.142510,2.596515,-0.134590,2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0
